# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew
import os
from langchain_openai import ChatOpenAI

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [13]:
os.environ["OPENAI_API_KEY"] = 'dummy_key'
os.environ["SERPER_API_KEY"]  = '56f516fc79785163aa408a550a17fa908e5efa19' # you can get free serper api keys from serper.dev. Just sign up and copy here.

llm = ChatOpenAI(
    model = 'ollama/llama3.3:latest',
    base_url="http://localhost:11434/v1"
)

## crewAI Tools

In [14]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [15]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [16]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [17]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [18]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [19]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [20]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [21]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [22]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [23]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [24]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 589db1ce-75fd-47ad-82d2-e1f1d8dc0c61                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task: Can you provide me with historical stock prices and other relevant data for AAPL?                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Thought: Thought: To provide historical stock prices and other relevant data for AAPL, I need to find a        │
│  reliable source of financial information that can be accessed through the tools at my disposal. Since I have   │
│  access to the internet, I should first try searching for websites that offer free or subscription-based        │
│  financial data.                                                                                                │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"website_url\": \"https://finance.yahoo.com/quote/AAPL/history\"}"                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'AAPL news', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic':           │
│  [{'title': 'Apple Inc. (AAPL) - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/AAPL/', 'snippet':    │
│  'Investors are cooling on Apple (AAPL) as other tech giants race ... news on 6/23/25 and open mixed to         │
│  slightly higher. The U.S. stock ...', 'position': 1, 'sitelinks': [{'title': 'News', 'link':                   │
│  'https://finance.yahoo.com/quote/AAPL/news/'}, {'title': 'Statistics', 'link':                                 │
│  'https://finance.yahoo.com/quote/AAPL/key-statistics/'}, {'title': 'Chart', 'link':                            │
│  'https://finance.yahoo.com/quote/AAPL/chart/'}, {'title': 'Yahoo Finance Historical Prices', 'link':           │
│  'https://finance.yahoo.com/quote/AAPL/history/'}]}, {'title': 'AAPL: Apple Inc - Stock Price, Quote and News   │
│  - CNBC', 'link': 'https://www.cnbc.com/quotes/AAPL', 'snippet': 'Get Apple Inc (AAPL:NASDAQ) real-time stock   │
│  quotes, news, price and financial information from CNBC.', 'position': 2}, {'title': 'Apple Inc. (AAPL)        │
│  Latest Stock News & Headlines - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/AAPL/news/',          │
│  'snippet': 'Get the latest Apple Inc. (AAPL) stock news and headlines to help you in your trading and          │
│  investing decisions.', 'position': 3}, {'title': 'AAPL Stock Quote Price and Forecast - CNN', 'link':          │
│  'https://www.cnn.com/markets/stocks/AAPL', 'snippet': 'View Apple Inc. AAPL stock quote prices, financial      │
│  information, real-time forecasts, and company news from CNN.', 'position': 4}, {'title': 'Apple (AAPL) News    │
│  Headlines | Nasdaq', 'link': 'https://www.nasdaq.com/market-activity/stocks/aapl/news-headlines', 'snippet':   │
│  'Stay up-to-date on Apple Inc. Common Stock (AAPL) news with the latest updates, breaking headlines, news      │
│  articles, and more from around the web at ...', 'position': 5}, {'title': 'Investor Relations - Apple',        │
│  'link': 'https://investor.apple.com/investor-relations/default.aspx', 'snippet': 'Apple expands U.S. supply    │
│  chain with $500 million commitment. July 15, 2025. Today Ap...                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Risk Advisor                                                                                            │
│                                                                                                                 │
│  Thought: Thought: I have gathered some information about AAPL from the news search, including recent           │
│  developments and announcements that could impact its stock price. However, to analyze the potential risks      │
│  associated with each trading strategy, I need more specific information about the strategies themselves.       │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"website_url\": \"https://finance.yahoo.com/quote/AAPL/\"}"                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance                                           │
│  Oops, something went wrong News Today's news US Politics World Tech Audio Computing Gaming Wordle Home         │
│  entertainment TVs Phones Science Streaming Streaming reviews VPN Wearables Deals Prime Day 2025 Best Amazon    │
│  Prime Day deals More AI Apps AR and VR Business Cameras Cyber security Entertainment General Smart home        │
│  Social media Transportation Weather Climate change Health Wellness Mental health Sexual health Dermatology     │
│  Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements  │
│  Fitness Equipment Exercise Women’s health Sleep Healthy aging Hearing Mobility Science Originals The 360       │
│  Newsletters Games Life Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot      │
│  health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise  │
│  Women’s health Sleep Healthy aging Hearing Mobility Parenting Family health So mini ways Style and beauty It   │
│  Figures Unapologetically Horoscopes Shopping Style Accessories Clothing Luggage Shoes Beauty Hair Makeup       │
│  Skincare Sunscreen Health Dental Fitness Hair loss Hearing aids Mental health Mobility Nutrition Personal      │
│  care Sleep Women's health Home & Garden Bedding Cleaning Gardening Kitchen Outdoor Pets Tech Accessories       │
│  Audio Auto Computers Phones Smart home TVs Gift ideas Stores Amazon Best Buy Home Depot Macy's Nordstrom       │
│  Target Walmart Wayfair Shopping Guides Best non-toxic cutting boards Best heated socks Best body wash Best     │
│  cordless stick vacuums Best makeup removers Deals Father’s Day gifts Best Father’s Day gifts Best Father’s     │
│  Day gifts under $50 Prime Day 2025 Best Amazon Prime Day deals Food Travel Autos Gift ideas Buying guides      │
│  Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock  │
│  Market Originals Tariff Updates Newsletters Economies Earnings Tech Housing Crypto Mergers & IPOs Electric     │
│  Vehicle...                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

KeyboardInterrupt: 

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)